#### Libraries

In [33]:
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import xgboost as xgb
import catboost as ctb

from collections import defaultdict

In [2]:
pd.set_option('display.max_columns', 100)

#### Load Data

In [3]:
wd = '/Users/konstantinivanov/Documents/work/PIK/files-pik_digital_day'

In [4]:
train = pd.read_csv(wd + '/train.csv')
test = pd.read_csv(wd + '/test.csv')
#flat = pd.read_csv(wd + '/flat.csv', encoding='cp1251')
#price = pd.read_csv(wd + '/price.csv', encoding='cp1251')
#status = pd.read_csv(wd + '/status.csv', encoding='cp1251')

#### Matching bulk_id - gk_id

In [11]:
bulk_gk = flat[['id_bulk', 'id_gk']].drop_duplicates()

In [21]:
bulk_gk = bulk_gk.reset_index().drop(['index'], axis=1)

In [27]:
train_1 = pd.merge(train, bulk_gk, how='left', left_on=['bulk_id'], right_on=['id_bulk']).drop(['id_bulk'], axis=1)

In [37]:
test_1 = pd.merge(test, bulk_gk, how='left', left_on=['bulk_id'], right_on=['id_bulk']).drop(['id_bulk'], axis=1)

#### Unload data

In [43]:
data_dir = '/Users/konstantinivanov/Documents/work/PIK/files-pik_digital_day'
input_dir = '/Users/konstantinivanov/Documents/work/PIK/input'
output_dir = '/Users/konstantinivanov/Documents/work/PIK/output'
sub_dir = '/Users/konstantinivanov/Documents/work/PIK/sub'

In [44]:
train_1.drop(['start_square', 'plan_s', 'plan_m', 'plan_l', 'vid_0', 'vid_1', 'vid_2'], axis=1, inplace=True)
train_1.to_csv(input_dir + "/train_full.csv", index=False)

test_1['value'] = 0
test_1.to_csv(input_dir + "/test_.csv", index=False, columns = train_1.columns)

#### Matching start_square

Готовлю данные из января

In [54]:
start_square_01 = train[train['date1']=='2018-01-01'][['bulk_id', 'spalen', 'start_square', 'value']]
start_square_01['start_square_next'] = start_square_01['start_square'] - start_square_01['value']
start_square_01 = start_square_01.reset_index().drop(['index', 'start_square', 'value'], axis=1)

Готовлю тестовые данные из февраля

In [64]:
test_sub_02 = test[test['date1']=='2018-02-01']
test_sub_02 = pd.merge(test_sub_02, start_square_01, how='inner', on=['bulk_id', 'spalen'])
test_sub_02 = test_sub_02.rename(index=str, columns={"start_square_next": "start_square"})
test_sub_02['plan'] = test_sub_02['start_square']/test_sub_02['mean_sq']
test_sub_02['plan'] = test_sub_02['plan'].astype(int)

In [65]:
test_sub_02

,id,bulk_id,spalen,date1,price,mean_sq,mean_fl,month,month_cnt,Класс объекта,Количество помещений,Огорожена территория,Площадь земельного участка,Входные группы,Детский сад,Школа,Поликлиника,ФОК,Спортивная площадка,Автомойка,Кладовые,Колясочные,Кондиционирование,Вентлияция,Лифт,Система мусоротведения,Видеонаблюдение,Подземная парковка,Двор без машин,Машиномест,Площадь пром. зоны в радиусе 500 м,Площадь зеленой зоны в радиусе 500 м,До Кремля,До ТТК(км),До Садового(км),До большой дороги на машине(км),До удобной авторазвязки на машине(км),До метро пешком(км),До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,Курс,Cтавка по ипотеке,Вклады до 1 года,Вклады от 1 года до 3 лет,Вклады свыше 3 лет,start_square,plan
0,0,FF3814A9-9F7D-E711-8530-00505688958B,1,2018-02-01,144734.016393,37.97,9,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.70,1.40,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.2,6.16,8958.90,235
1,1,FF3814A9-9F7D-E711-8530-00505688958B,2,2018-02-01,136729.844961,57.78,9,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.70,1.40,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.2,6.16,14736.70,255
2,2,FF3814A9-9F7D-E711-8530-00505688958B,0,2018-02-01,167787.719298,23.21,10,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.70,1.40,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.2,6.16,1275.30,54
3,3,FF3814A9-9F7D-E711-8530-00505688958B,3,2018-02-01,132855.319149,79.39,9,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.70,1.40,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.2,6.16,7309.20,92
4,16,7726CF5A-CC0A-E511-8098-001EC9D56418,3,2018-02-01,182700.000000,93.60,3,2,38,комфорт,1094,нет,6.8,да,150,550,0,1,да,нет,да,да,1,1,1,контейнеры,1,да,да,871,4740.0,156205.0,13.0,8.0,11.0,0.69,7.00,0.35,0.45,0.40,0.54,4.0,8587.0,56.806705,9.78,5.48,6.2,6.16,187.20,2
5,19,72B1CAF6-A2DC-E511-9840-001EC9D56418,2,2018-02-01,106762.500000,58.52,10,2,38,комфорт,3958,нет,114.0,да,800,3150,600,1,да,нет,да,да,1,2,1,контейнеры,1,да,да,6000,13990.0,7000.0,20.0,14.0,17.0,1.80,6.70,5.80,0.23,1.20,1.60,7.0,7133.0,56.806705,9.78,5.48,6.2,6.16,127.50,2
6,20,72B1CAF6-A2DC-E511-9840-001EC9D56418,3,2018-02-01,101133.333333,93.20,18,2,38,комфорт,3958,нет,114.0,да,800,3150,600,1,да,нет,да,да,1,2,1,контейнеры,1,да,да,6000,13990.0,7000.0,20.0,14.0,17.0,1.80,6.70,5.80,0.23,1.20,1.60,7.0,7133.0,56.806705,9.78,5.48,6.2,6.16,279.60,3
7,21,35168781-8F60-E611-9523-001EC9D5643C,3,2018-02-01,95472.222222,93.56,8,2,38,комфорт,3958,нет,114.0,да,800,3150,600,1,да,нет,да,да,1,2,1,контейнеры,1,да,да,6000,13990.0,0.0,20.0,14.0,17.0,2.10,7.00,6.00,0.23,1.20,1.40,7.0,7133.0,56.806705,9.78,5.48,6.2,6.16,5052.10,53
8,22,35168781-8F60-E611-9523-001EC9D5643C,2,2018-02-01,95975.000000,67.96,9,2,38,комфорт,3958,нет,114.0,да,800,3150,600,1,да,нет,да,да,1,2,1,контейнеры,1,да,да,6000,13990.0,0.0,20.0,14.0,17.0,2.10,7.00,6.00,0.23,1.20,1.40,7.0,7133.0,56.806705,9.78,5.48,6.2,6.16,543.70,8
9,23,A8632090-8F60-E611-9523-001EC9D5643C,3,2018-02-01,94876.666667,90.47,12,2,38,комфорт,3958,нет,114.0,да,800,3150,600,1,да,нет,да,да,1,2,1,контейнеры,1,да,да,6000,13990.0,0.0,20.0,14.0,17.0,2.30,7.20,6.00,0.23,1.20,1.30,7.0,7133.0,56.806705,9.78,5.48,6.2,6.16,2714.10,30


Готовлю тренировочные данные

In [67]:
train['plan'] = train['plan_s'] + train['plan_m'] + train['plan_l']

#### Unload Data

In [68]:
data_dir = '/Users/konstantinivanov/Documents/work/PIK/files-pik_digital_day'
input_dir = '/Users/konstantinivanov/Documents/work/PIK/input'
output_dir = '/Users/konstantinivanov/Documents/work/PIK/output'
sub_dir = '/Users/konstantinivanov/Documents/work/PIK/sub'

In [69]:
train.drop(['plan_s', 'plan_m', 'plan_l', 'vid_0', 'vid_1', 'vid_2'], axis=1, inplace=True)
train.to_csv(input_dir + "/train_full.csv", index=False)

test_sub_02['value'] = 0
test_sub_02.to_csv(input_dir + "/test_.csv", index=False, columns = train.columns)

In [73]:
test

,id,bulk_id,spalen,date1,price,mean_sq,mean_fl,month,month_cnt,Класс объекта,Количество помещений,Огорожена территория,Площадь земельного участка,Входные группы,Детский сад,Школа,Поликлиника,ФОК,Спортивная площадка,Автомойка,Кладовые,Колясочные,Кондиционирование,Вентлияция,Лифт,Система мусоротведения,Видеонаблюдение,Подземная парковка,Двор без машин,Машиномест,Площадь пром. зоны в радиусе 500 м,Площадь зеленой зоны в радиусе 500 м,До Кремля,До ТТК(км),До Садового(км),До большой дороги на машине(км),До удобной авторазвязки на машине(км),До метро пешком(км),До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,Курс,Cтавка по ипотеке,Вклады до 1 года,Вклады от 1 года до 3 лет,Вклады свыше 3 лет
0,0,FF3814A9-9F7D-E711-8530-00505688958B,1,2018-02-01,144734.016393,37.97,9,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.20,6.16
1,1,FF3814A9-9F7D-E711-8530-00505688958B,2,2018-02-01,136729.844961,57.78,9,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.20,6.16
2,2,FF3814A9-9F7D-E711-8530-00505688958B,0,2018-02-01,167787.719298,23.21,10,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.20,6.16
3,3,FF3814A9-9F7D-E711-8530-00505688958B,3,2018-02-01,132855.319149,79.39,9,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.20,6.16
4,4,FF3814A9-9F7D-E711-8530-00505688958B,1,2018-03-01,147387.111111,37.99,9,3,39,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.063605,9.70,5.53,6.13,5.63
5,5,FF3814A9-9F7D-E711-8530-00505688958B,2,2018-03-01,138440.772532,57.80,9,3,39,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.063605,9.70,5.53,6.13,5.63
6,6,FF3814A9-9F7D-E711-8530-00505688958B,0,2018-03-01,171589.583333,23.67,10,3,39,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.063605,9.70,5.53,6.13,5.63
7,7,FF3814A9-9F7D-E711-8530-00505688958B,3,2018-03-01,134381.707317,80.46,8,3,39,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.063605,9.70,5.53,6.13,5.63
8,8,FF3814A9-9F7D-E711-8530-00505688958B,1,2018-04-01,153177.777778,38.00,9,4,40,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,60.769891,9.59,5.25,6.25,5.35
9,9,FF3814A9-9F7D-E711-8530-00505688958B,2,2018-04-01,143370.813397,57.79,9,4,40,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,60.769891,9.59,5.25,6.25,5.35
